https://www.adaltas.com/en/2021/03/05/tfx-overview/

In [ ]:
import os
import pprint
import tempfile
import urllib
import tensorboard

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [ ]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Vitals Pipeline example.
_vitals_root = os.path.join(_tfx_root, '../pipeline/vitals')

# This is the path where your model will be pushed for serving.
_serving_model_dir = '../models/vitals_simple'

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

In [ ]:
_example_root = '../data/new'
_example_raw = "../data/raw/pat_vitals_unlabelled-dataSepsis.csv"
#s3.download_file(s3_bucket, "data/pat_vitals_unlabelled-dataSepsis.csv", _data_raw)

In [ ]:
# view the actual patient data with sepsis labels
# 1 = septic, 0 = not-septic
!head {_example_raw}

In [ ]:
context = InteractiveContext(pipeline_root='../pipeline')

In [1]:
# input_config An example_gen_pb2.Input instance, providing input configuration. If unset, the files under input_base will be treated as a single split.
# output_config An example_gen_pb2.Output instance, providing output configuration. If unset, default splits will be 'train' and 'eval' with size 2:1
# range_config An optional range_config_pb2.RangeConfig instance, specifying the range of span values to consider. If unset, driver will default to searching for latest span with no restrictions.
# outputs Component's output channel dict.

inference_example_gen = tfx.components.CsvExampleGen(
    # input_base an external directory containing the CSV files.
    input_base=_example_root,
    # name of the root folder the TFRecord is output to ('examples' by default)
    instance_name=inference
    )
context.run(inference_example_gen, enable_cache=True)

NameError: name 'tfx' is not defined

In [ ]:
# Uses BulkInferrer to inference on examples.
# examples A BaseChannel of type standard_artifacts.Examples, usually produced by an ExampleGen component. required
# model A BaseChannel of type standard_artifacts.Model, usually produced by a Trainer component.

bulk_inferrer = BulkInferrer(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'])

bulk_inferrer = BulkInferrer(
 examples=inference_example_gen.outputs[‘examples’],
 model_export=trainer.outputs[‘output’],
 model_blessing=evaluator.outputs[‘blessing’],
 data_spec=bulk_inferrer_pb2.DataSpec(
 example_splits=[‘unlabelled’]),
 model_spec=bulk_inferrer_pb2.ModelSpec())

context.run(bulk_inferrer)